# RAG: Embeddings locales (Transformers) + Pinecone + Gemini 

Mejores prácticas: chunking con solapamiento, MMR, citas y orquestación LCEL.

Crea `.env` en  la ruta del `RAG` con:

- `GOOGLE_API_KEY=...` (obligatorio, para respuestas con Gemini)
- `PINECONE_API_KEY=...` (obligatorio, para índices/vector store)
- `PINECONE_REGION=us-east-1` (opcional, por defecto `us-east-1`)
- `PC_INDEX_NAME=...` (obligatorio, usar minúsculas y `-`)
- `PC_NAMESPACE=...` (obligatorio, usar minúsculas y `-`)
- `HF_EMBED_MODEL=intfloat/multilingual-e5-base` (opcional, embeddings locales)
- `PDF_PATH=...` (obligatorio, ruta al PDF)
- `CHUNK_SIZE=1000`, `CHUNK_OVERLAP=150` (opcional)
- `PC_RESET_IF_DIM_MISMATCH=false` (opcional: `true` para recrear índice si la dimensión no coincide)

Nota: No se usan alternativas; si faltan claves obligatorias, se detiene con mensaje claro. Ademas comparti el `.env.example` para referencia.

In [ ]:
%pip install -q -U "langchain>=0.2.12" langchain-community "langchain-google-genai>=3.0.0" langchain-text-splitters "pinecone-client>=3.0.0" langchain-pinecone python-dotenv pypdf transformers sentence-transformers "huggingface_hub[hf_xet]" langchain_huggingface

## BLOQUE 1 — Configuración y entorno
- Importa `os`, `re` y `dotenv.load_dotenv` para cargar variables desde `.env`.
- Define parámetros clave: `GOOGLE_API_KEY`, `PINECONE_API_KEY`, `HF_EMBED_MODEL`, `INDEX_NAME`, `NAMESPACE`, `PDF_PATH`, `CHUNK_SIZE`, `CHUNK_OVERLAP`.
- Realiza `assert` sobre claves obligatorias para fallar temprano si faltan.
- Motivo: centraliza configuración y asegura que el entorno tenga credenciales y parámetros antes de continuar.

In [ ]:
import os, re
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_REGION = os.getenv("PINECONE_REGION", "us-east-1")
HF_EMBED_MODEL = os.getenv("HF_EMBED_MODEL", "intfloat/multilingual-e5-base")
INDEX_NAME = os.getenv("PC_INDEX_NAME", " ")
NAMESPACE = os.getenv("PC_NAMESPACE", " ")
PDF_PATH = os.getenv("PDF_PATH", r" ")
CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "1000"))
CHUNK_OVERLAP = int(os.getenv("CHUNK_OVERLAP", "150"))
PC_RESET_IF_DIM_MISMATCH = os.getenv("PC_RESET_IF_DIM_MISMATCH", "false").lower() == "true"

assert GOOGLE_API_KEY, "Falta GOOGLE_API_KEY en .env (Gemini es obligatorio)."
assert PINECONE_API_KEY, "Falta PINECONE_API_KEY en .env (Pinecone es obligatorio)."

print("GOOGLE_API_KEY: OK")
print("PINECONE_API_KEY: OK")
print("PINECONE_REGION:", PINECONE_REGION)
print("HF_EMBED_MODEL:", HF_EMBED_MODEL)
print("INDEX_NAME:", INDEX_NAME, "NAMESPACE:", NAMESPACE)
print("PDF_PATH:", PDF_PATH)
print("CHUNK_SIZE:", CHUNK_SIZE, "CHUNK_OVERLAP:", CHUNK_OVERLAP)
print("PC_RESET_IF_DIM_MISMATCH:", PC_RESET_IF_DIM_MISMATCH)

## Embeddings locales (Transformers / Sentence-Transformers)
- Importa `HuggingFaceEmbeddings` desde `langchain_huggingface`.
- Crea `doc_emb` con el modelo indicado y normalización de embeddings.
- Calcula `detected_dim` (dimensión del vector), útil para configurar el índice de Pinecone.
- Motivo: obtener representaciones vectoriales consistentes para consultas y documentos (base del RAG).

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

doc_emb = HuggingFaceEmbeddings(
    model_name=HF_EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True}
)
detected_dim = len(doc_emb.embed_query("prueba dimension"))
print(f"Embeddings locales cargados: {HF_EMBED_MODEL} (dim={detected_dim})")

## Ingesta de PDF y chunking con solapamiento (mejores prácticas)
- Importa `PdfReader` (`pypdf`) y `RecursiveCharacterTextSplitter`.
- Extrae texto por página, crea metadatos (`source`, `page`) y realiza chunking con solapamiento.
- Produce `docs` (lista de `Document` con contenido y metadatos).
- Motivo: preparar el corpus en fragmentos manejables que capturen contexto y mejoren la recuperación.

In [ ]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

reader = PdfReader(PDF_PATH)
pages = [page.extract_text() or "" for page in reader.pages]
texts, metas = [], []
for i, t in enumerate(pages, start=1):
    t = (t or "").strip()
    if not t:
        continue
    texts.append(t)
    metas.append({"source": os.path.basename(PDF_PATH), "page": i})

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs = splitter.create_documents(texts, metadatas=metas)
print(f"Total de chunks: {len(docs)} (chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")

## Inicializar Pinecone e índice (sanitización y validación de dimensión)
- Importa `Pinecone` (y `ServerlessSpec`), normaliza el nombre del índice y lo crea/valida.
- Alinea la dimensión del índice con `detected_dim` y muestra el namespace en uso.
- Motivo: disponer de un almacén vectorial escalable para búsquedas semánticas con embeddings.

In [ ]:
from pinecone import Pinecone, ServerlessSpec

def sanitize_index_name(name: str) -> str:
    s = name.lower()
    s = re.sub(r'[^a-z0-9-]', '-', s)
    s = re.sub(r'-{2,}', '-', s)
    s = s.strip('-')
    return s or 'index'

EMBED_DIM = detected_dim
pc = Pinecone(api_key=PINECONE_API_KEY)
safe_index = sanitize_index_name(INDEX_NAME)
if safe_index != INDEX_NAME:
    print(f"Renombrando índice inválido '{INDEX_NAME}' → '{safe_index}'")
    INDEX_NAME = safe_index

existing = [x.name for x in pc.list_indexes()]
if INDEX_NAME not in existing:
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBED_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=PINECONE_REGION),
    )
    print(f"Índice creado: {INDEX_NAME} (dim={EMBED_DIM}, región={PINECONE_REGION})")
else:
    desc = pc.describe_index(INDEX_NAME)
    idx_dim = getattr(desc, "dimension", EMBED_DIM)
    if idx_dim != EMBED_DIM:
        print(f"Advertencia: dimensión del índice ({idx_dim}) != embeddings ({EMBED_DIM}).")
        if PC_RESET_IF_DIM_MISMATCH:
            print("Recreando índice para alinear dimensión...")
            pc.delete_index(INDEX_NAME)
            pc.create_index(
                name=INDEX_NAME,
                dimension=EMBED_DIM,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region=PINECONE_REGION),
            )
            print(f"Índice recreado: {INDEX_NAME} (dim={EMBED_DIM})")

index = pc.Index(INDEX_NAME)
print(f"Usando índice Pinecone: {INDEX_NAME} (namespace='{NAMESPACE}')")

## Upsert de chunks en Pinecone (metadatos e ids estables)
- Importa `PineconeVectorStore` y construye listas de `texts`, `metadatas` e `ids`.
- Crea `vectorstore` y realiza `add_texts` para subir los chunks al índice.
- Motivo: persistir los vectores de los documentos en Pinecone como base del retriever.

In [33]:
from langchain_pinecone import PineconeVectorStore
texts = [d.page_content for d in docs]
metadatas = [d.metadata for d in docs]
ids = [f"{m['page']}-{i}" for i, m in enumerate(metadatas)]
vectorstore = PineconeVectorStore(index_name=INDEX_NAME, embedding=doc_emb, namespace=NAMESPACE)
_ = vectorstore.add_texts(texts=texts, metadatas=metadatas, ids=ids)
print(f"Upsert en Pinecone completado: {len(texts)} chunks")

Upsert en Pinecone completado: 101 chunks


## Retriever MMR y compresión manual basada en embeddings
- Importa `numpy`, crea `retriever` con `MMR` para diversidad de resultados.
- Define `_cosine` y la clase `SimpleCompressionRetriever` (reranking por similitud coseno: consulta vs documentos).
- Crea `cretriever` con `similarity_threshold` y `top_k`, e imprime la configuración activa.
- Motivo: recuperar candidatos con MMR y filtrar/reranquear manualmente para maximizar relevancia del contexto.

In [5]:
from langchain_pinecone import PineconeVectorStore
import numpy as np

if 'vectorstore' not in globals():
    vectorstore = PineconeVectorStore(index_name=INDEX_NAME, embedding=doc_emb, namespace=NAMESPACE)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 24, "fetch_k": 64, "lambda_mult": 0.4},
)

def _cosine(a, b):
    a = np.array(a, dtype=np.float32)
    b = np.array(b, dtype=np.float32)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0.0 or nb == 0.0:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

class SimpleCompressionRetriever:
    def __init__(self, base_retriever, embeddings, similarity_threshold=0.65, top_k=6):
        self.base_retriever = base_retriever
        self.embeddings = embeddings
        self.similarity_threshold = similarity_threshold
        self.top_k = top_k

    def get_relevant_documents(self, query):
        docs = (
            self.base_retriever.get_relevant_documents(query)
            if hasattr(self.base_retriever, "get_relevant_documents")
            else self.base_retriever.invoke(query)
        )
        qv = self.embeddings.embed_query(query)
        dvs = self.embeddings.embed_documents([d.page_content for d in docs])
        scored = []
        for d, v in zip(docs, dvs):
            s = _cosine(v, qv)
            scored.append((s, d))
        filtered = [d for s, d in sorted(scored, key=lambda x: x[0], reverse=True) if s >= self.similarity_threshold]
        if not filtered:
            filtered = [d for _, d in sorted(scored, key=lambda x: x[0], reverse=True)[: self.top_k]]
        return filtered[: self.top_k]

    def invoke(self, query):
        return self.get_relevant_documents(query)

cretriever = SimpleCompressionRetriever(retriever, doc_emb, similarity_threshold=0.65, top_k=6)
print(f"Compresión contextual (manual) habilitada: umbral={cretriever.similarity_threshold}, top_k={cretriever.top_k}")

Compresión contextual (manual) habilitada: umbral=0.65, top_k=6


## Cadena RAG (LCEL) con Gemini y citas
- Importa `ChatGoogleGenerativeAI`, `ChatPromptTemplate`, `StrOutputParser`, `RunnablePassthrough`, `RunnableLambda`.
- Define `format_docs` para construir el contexto con citas `[p. N]` y el `SYSTEM_PROMPT` con instrucciones claras.
- Crea `prompt`, `llm` (`gemini-2.0-flash`, `temperature=0.2`) y el grafo LCEL: `{context, question} -> prompt -> llm -> parser`.
- Motivo: orquestar el flujo RAG de forma declarativa con LCEL, garantizando formato y trazabilidad del contexto.

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def format_docs(docs):
    parts = []
    for d in docs:
        pg = d.metadata.get("page")
        src = d.metadata.get("source")
        parts.append(f"[p. {pg}] ({src})\n{d.page_content}")
    return "\n\n".join(parts)

SYSTEM_PROMPT = (
    "Eres un asistente experto en recuperación de información. Responde en español, claro y estructurado.\n"
    "Usa la información del 'Contexto' para contestar con precisión.\n"
    "Incluye citas de página en formato [p. N] cuando corresponda. Si falta información, indícalo."
)
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "Pregunta: {question}\n\nContexto:\n{context}"),
])

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

retrieval = RunnableLambda(lambda q: cretriever.invoke(q))
format_ctx = RunnableLambda(format_docs)

rag_chain = (
    {"context": (retrieval | format_ctx), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("Cadena RAG con Gemini lista.")

Cadena RAG con Gemini lista.


## Consulta de ejemplo con citas
- Define `question`, ejecuta `rag_chain.invoke(question)` y muestra la respuesta.
- Recupera `docs_used` desde `cretriever` y lista las páginas citadas.
- Motivo: demostrar el flujo completo, verificar páginas usadas y evaluar la calidad de la recuperación.

In [7]:
question = (
    "que dicen las primeras paginas?"
)

answer = rag_chain.invoke(question)
print(answer)

docs_used = cretriever.invoke(question)
pages = sorted({d.metadata.get("page") for d in docs_used if d.metadata.get("page")})
print("Citas (páginas) usadas:", pages)

La primera página del documento indica que la conclusión se encuentra en la página 14 [p. 1].

La página 2 explica que la mayoría de lo que se discute bajo el tema de la IA se refiere al aumento de la automatización de tareas mediante el uso del aprendizaje automático y la toma de decisiones automatizada. En el centro de los debates actuales sobre la IA y las aplicaciones de aprendizaje automático se encuentra el uso de algoritmos, que son reglas seguidas por una computadora, programadas por humanos, que traducen los datos de entrada en salidas [p. 2].
Citas (páginas) usadas: [1.0, 2.0, 17.0, 18.0, 20.0]
